In [29]:
def get_chunks(path):
    headers_to_split_on = [
        ('#', 'Header 1'),
        ('##', 'Header 2'),
        ('###', 'Header 3'),
        # ('###', 'Header 4')
    ]
    all_chunks = []
    for root, _, files in os.walk(path):
        for file in tqdm(files):
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                try:
                    # Загрузка файла
                    with open(file_path) as f:
                        markdown_content = f.read()
                        
                    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
                    doc_chunks = markdown_splitter.split_text(markdown_content)
                    for chunk in doc_chunks:
                        all_chunks.append((list(chunk.metadata.values())[0], chunk.page_content))
                   
    
                except Exception as e:
                    print(f'Ошибка обработки файла {file_path}: {e}')
                    continue
    return all_chunks
                
        

In [1]:
from langchain_gigachat import chat_models, embeddings, GigaChat, GigaChatEmbeddings, output_parsers, tools, utils


In [ ]:
GigaChatEmbeddings()

In [2]:
from sentence_transformers import SentenceTransformer
import os

# Попробуйте увеличить таймаут (в секундах)
os.environ['HF_TRANSFER_TIMEOUT'] = '100'  # Увеличиваем таймаут для HF
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '600'  # Увеличиваем таймаут загрузки

embedding_model = SentenceTransformer(
    model_name_or_path='sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    # Дополнительные опции для requests
    request_timeout=120
)
test_embedding = embedding_model.embed_query("FastAPI is a modern web frameworffk")
print(f"Размерность эмбеддинга: {len(test_embedding)}")

TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'request_timeout'

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
# Инициализация модели для создания эмбеддингов
# Хорошая модель для старта: all-MiniLM-L6-v2
embedding_model = HuggingFaceEmbeddings(
    # model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_name = 'intfloat/multilingual-e5-large',
    model_kwargs={'device': 'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}  # Нормализация для косинусного сходства
)

# Тестируем модель
test_embedding = embedding_model.embed_query("FastAPI is a modern web frameworffk")
print(f"Размерность эмбеддинга: {len(test_embedding)}")

TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'model_name'

In [7]:
client = QdrantClient(url='localhost:6333')  # Используйте "localhost:6333" для standalone-сервера

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain.vectorstores import Qdrant

# Локальный режим (для тестирования)
client = QdrantClient(":memory:")  # Используйте "localhost:6333" для standalone-сервера

# Или подключение к Qdrant Cloud
# client = QdrantClient(
#     url="your-cluster-url",
#     api_key="your-api-key",
# )

# Создание коллекции (если её нет)
try:
    client.create_collection(
        collection_name="fastapi_docs",
        vectors_config=VectorParams(
            size=384,  # Должно соответствовать размерности выбранной модели эмбеддингов!
            distance=Distance.COSINE  # Косинусное расстояние для семантического сходства
        )
    )
except Exception as e:
    print(f"Коллекция可能 уже существует: {e}")

# Инициализация векторного хранилища LangChain для Qdrant
vector_store = Qdrant(
    client=client,
    collection_name="fastapi_docs",
    embeddings=embedding_model,
)

NameError: name 'embedding_model' is not defined

In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain.vectorstores import Qdrant

# Локальный режим (для тестирования)
client = QdrantClient(":memory:")  # Используйте "localhost:6333" для standalone-сервера

# Или подключение к Qdrant Cloud
# client = QdrantClient(
#     url="your-cluster-url",
#     api_key="your-api-key",
# )

# Создание коллекции (если её нет)
try:
    client.create_collection(
        collection_name="fastapi_docs",
        vectors_config=VectorParams(
            size=384,  # Должно соответствовать размерности выбранной модели эмбеддингов!
            distance=Distance.COSINE  # Косинусное расстояние для семантического сходства
        )
    )
except Exception as e:
    print(f"Коллекция可能 уже существует: {e}")

# Инициализация векторного хранилища LangChain для Qdrant
vector_store = Qdrant(
    client=client,
    collection_name="fastapi_docs",
    embeddings=embedding_model,
)

NameError: name 'embedding_model' is not defined

In [ ]:
# Пример запроса
query = "How to create a POST endpoint in FastAPI?"
found_docs = vector_store.similarity_search(
    query=query,
    k=5  # Количество возвращаемых результатов
)

# Вывод результатов
for i, doc in enumerate(found_docs):
    print(f"Результат #{i+1}:")
    print(f"Контент: {doc.page_content[:200]}...")
    print(f"Источник: {doc.metadata.get('source', 'N/A')}")
    print(f"Рейтинг: {score}")  # Если используется similarity_search_with_score
    print("-" * 50)

In [2]:
import os
from langchain.docstore.document import  Document
from langchain.text_splitter import MarkdownHeaderTextSplitter
from tqdm import tqdm
from typing import List, Tuple
PATH = './docs/'

def get_chunks(path) -> List[Tuple[str, str]]:
    headers_to_split_on = [
        ('#', 'Header 1'),
        ('##', 'Header 2'),
        ('###', 'Header 3'),
        # ('###', 'Header 4')
    ]

    all_chunks = []
    for root, _, files in os.walk(path):
        for file in tqdm(files):
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                try:
                    # Загрузка файла
                    with open(file_path) as f:
                        markdown_content = f.read()
                        
                    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
                    doc_chunks = markdown_splitter.split_text(markdown_content)
                    
                    for chunk in doc_chunks:
                        all_chunks.append((list(chunk.metadata.values())[0], chunk.page_content)) # на выходе список кортежей вида (заголовок, текст)
                   
                except Exception as e:
                    print(f'Ошибка обработки файла {file_path}: {e}')
                    continue

    print(f'Всего создано {len(all_chunks)} чанков из {len(files)} файлов')
    return all_chunks
                

# def chunk_cleaner(chunk):
    

In [6]:
c = get_chunks(PATH)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 197.48it/s]

Всего создано 709 чанков из 5 файлов


In [10]:
c[3]

('Альтернативы, источники вдохновения и сравнения',
 '### <a href="https://www.django-rest-framework.org/" class="external-link" target="_blank">Django REST Framework</a>  \nФреймворк Django REST был создан, как гибкий инструментарий для создания веб-API на основе Django.  \nDRF использовался многими компаниями, включая Mozilla, Red Hat и Eventbrite.  \nЭто был один из первых примеров **автоматического документирования API** и это была одна из первых идей, вдохновивших на создание **FastAPI**.  \n/// note | Заметка  \nDjango REST Framework был создан Tom Christie.\nОн же создал Starlette и Uvicorn, на которых основан **FastAPI**.  \n///  \n/// check | Идея для **FastAPI**  \nДолжно быть автоматическое создание документации API с пользовательским веб-интерфейсом.  \n///')

In [34]:
import re
def clean_text(text: str) -> str:
    '''Чистка текста от HTML-тегов, лишних символов и сусора с сохранением структуры и содержания'''

    clean_text = text.strip() 
    # Удаляем теги HTML-теги
    clean_text = re.sub(r'<a\s+href="([^"]*)"[^>]*>([^<]*)</a>', r'\2 (\1)', clean_text)
    clean_text = re.sub(r'///\s*\w+\s*|.*?///', '', clean_text, re.DOTALL) # Символы типа '///'
    clean_text = re.sub(r'#+\s*', '', clean_text)  # Заголовки
    return clean_text

In [33]:
t = '### <a href="https://www.django-rest-framework.org/" class="external-link" target="_blank">Django REST Framework</a>  \nФреймворк Django REST был создан, как гибкий инструментарий для создания веб-API на основе Django.  \nDRF использовался многими компаниями, включая Mozilla, Red Hat и Eventbrite.  \nЭто был один из первых примеров **автоматического документирования API** и это была одна из первых идей, вдохновивших на создание **FastAPI**.  \n/// note | Заметка  \nDjango REST Framework был создан Tom Christie.\nОн же создал Starlette и Uvicorn, на которых основан **FastAPI**.  \n///  \n/// check | Идея для **FastAPI**  \nДолжно быть автоматическое создание документации API с пользовательским веб-интерфейсом.  \n///'
clean_text(t)

'Django REST Framework (https://www.django-rest-framework.org/)  \nФреймворк Django REST был создан, как гибкий инструментарий для создания веб-API на основе Django.  \nDRF использовался многими компаниями, включая Mozilla, Red Hat и Eventbrite.  \nЭто был один из первых примеров **автоматического документирования API** и это была одна из первых идей, вдохновивших на создание **FastAPI**.  \n| Заметка  \nDjango REST Framework был создан Tom Christie.\nОн же создал Starlette и Uvicorn, на которых основан **FastAPI**.  \n  \n| Идея для **FastAPI**  \nДолжно быть автоматическое создание документации API с пользовательским веб-интерфейсом.  \n'

In [93]:
import os
import re
from langchain.docstore.document import  Document
from langchain.text_splitter import MarkdownHeaderTextSplitter
from tqdm import tqdm
from typing import List, Tuple
PATH = './docs/'

def get_chunks(path) -> List[Tuple[str, str]]:
    headers_to_split_on = [
        ('#', 'Header 1'),
        ('##', 'Header 2'),
        ('###', 'Header 3'),
    ]

    all_chunks = []
    for root, _, files in os.walk(path):
        for file in tqdm(files):
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                try:
                    # Загрузка файла
                    with open(file_path, 'r', encoding='utf-8') as f:
                        markdown_content = f.read()
                        
                    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
                    doc_chunks = markdown_splitter.split_text(markdown_content)
                    
                    for chunk in doc_chunks:                        
                        content = clean_text(chunk.page_content) # Чистим текст
                        all_chunks.append((list(chunk.metadata.values())[0], content)) # на выходе список кортежей вида (заголовок, текст)
                   
                except Exception as e:
                    print(f'Ошибка обработки файла {file_path}: {e}')
                    continue

    print(f'Всего создано {len(all_chunks)} чанков из {len(files)} файлов')
    return all_chunks
                


def clean_text(text: str) -> str:
    '''Чистка текста от HTML-тегов, лишних символов и сусора с сохранением структуры и содержания'''
    
    if not text or not text.strip():
        return ""

    cleaned = text.strip() 

    # Удаляем теги HTML-теги
    cleaned = re.sub(r'<a\s+href="([^"]*)"[^>]*>([^<]*)</a>', r'\2 (\1)', cleaned)
    cleaned = re.sub(r'<[^>]+>', '', cleaned)

    # Символы типа '///'
    cleaned = re.sub(r'///.*?///', '', cleaned, flags=re.DOTALL)

    # Заголовки
    cleaned = re.sub(r'#+\s*', '', cleaned)
    
    # Жирный текст
    cleaned = re.sub(r'\*\*(.*?)\*\*', r'\1', cleaned) 
    
    # Курсив
    cleaned = re.sub(r'\*(.*?)\*', r'\1', cleaned)  
    
    # Удаляем лишние пробелы и переносы строк
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = cleaned.strip()

    return cleaned
    

In [96]:
c = get_chunks('./docs/deployment/')

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 167.22it/s]

Всего создано 96 чанков из 6 файлов


In [99]:
c

[('Концепции развёртывания',
  'Концепции развёртывания Существует несколько концепций, применяемых для развёртывания приложений FastAPI, равно как и для любых других типов веб-приложений, среди которых вы можете выбрать наиболее подходящий способ. Самые важные из них: * Использование более безопасного протокола HTTPS * Настройки запуска приложения * Перезагрузка приложения * Запуск нескольких экземпляров приложения * Управление памятью * Использование перечисленных функций перед запуском приложения. Рассмотрим ниже влияние каждого из них на процесс развёртывания. Наша конечная цель - обслуживать клиентов вашего API безопасно и бесперебойно, с максимально эффективным использованием вычислительных ресурсов (например, удалённых серверов/виртуальных машин). 🚀 Здесь я немного расскажу Вам об этих концепциях и надеюсь, что у вас сложится интуитивное понимание, какой способ выбрать при развертывании вашего API в различных окружениях, возможно, даже ещё не существующих. Ознакомившись с этими 

In [76]:
clean_text(c[:10][3][1])

'Django REST Framework (https://www.django-rest-framework.org/) Фреймворк Django REST был создан, как гибкий инструментарий для создания веб-API на основе Django. DRF использовался многими компаниями, включая Mozilla, Red Hat и Eventbrite. Это был один из первых примеров автоматического документирования API и это была одна из первых идей, вдохновивших на создание FastAPI.'

In [78]:
c

[('Альтернативы, источники вдохновения и сравнения',
  '# Альтернативы, источники вдохновения и сравнения  \nЧто вдохновило на создание **FastAPI**, сравнение его с альтернативами и чему он научился у них.'),
 ('Альтернативы, источники вдохновения и сравнения',
  '## Введение  \n**FastAPI** не существовал бы, если б не было более ранних работ других людей.  \nОни создали большое количество инструментов, которые вдохновили меня на создание **FastAPI**.  \nЯ всячески избегал создания нового фреймворка в течение нескольких лет.\nСначала я пытался собрать все нужные функции, которые ныне есть в **FastAPI**, используя множество различных фреймворков, плагинов и инструментов.  \nНо в какой-то момент не осталось другого выбора, кроме как создать что-то, что предоставляло бы все эти функции сразу.\nВзять самые лучшие идеи из предыдущих инструментов и, используя новые возможности Python (которых не было до версии 3.6, то есть подсказки типов), объединить их.'),
 ('Альтернативы, источники вдохно